# Video Collaboration
### All images used will be RGB Images in every function

In [1]:
# Importing Libraries
import cv2
import numpy as np
import glob
import natsort
from natsort import natsorted

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
from tqdm import tqdm

%matplotlib inline

### Video to Image Converter

In [2]:
def vid2frame(video):
    name = video
    name = name[:-4]
    save_dir = name + "_frames"
    print(save_dir)
    print("Creating a folder if it is not present\n")
    # Folder creation for frames
    CHECK_FOLDER = os.path.isdir(save_dir)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(save_dir)
        print("Created the folder!")

    # Opens the Video file
    cap = cv2.VideoCapture(video)
    i = 0
    print("Converting to Frames....")
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        cv2.imwrite(save_dir + "/" + str(i) + '.jpg', frame)
        i += 1

    cap.release()
    cv2.destroyAllWindows()
    
    print("Video to frame Conversion Complete.")
    
    return save_dir

In [3]:
vid2frame("index.mp4")

index_frames
Creating a folder if it is not present

Created the folder!
Converting to Frames....
Video to frame Conversion Complete.


'index_frames'

### Frames to Video Converter

In [3]:
def frame2vid(edited_frames_dir, fps):
    vid_name = edited_frames_dir + "_" + str(fps)
    
    frames = glob.glob(edited_frames_dir + "/" + "*.jpg")
    frames = natsorted(frames)
    img_array = []
    print("Joining frames...")
    for frame in tqdm(frames):
        img = cv2.imread(frame)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

    out = cv2.VideoWriter(edited_frames_dir + "_fps_" + str(fps) + ".avi", cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    print("Saving to a video...")
    for i in tqdm(range(len(img_array))):
        out.write(img_array[i])
    out.release()
    print("Video Saved")

# Various Image Processing techniques 
# -> Effects

In [121]:
# Canny Edge
def Canny(image):
    # Defining canny edge detection threshold parameters
    low_thresh = 100
    high_thresh = 150
    edges = cv2.Canny(image, 100, 150)
    return edges

In [122]:
# Edit this function to create your own pipeline.
def colorBinary(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return color_binary

In [123]:
def binaryImage(image):
    thresh = (1, 255)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    binary = np.zeros_like(gray)
    binary[(gray > thresh[0]) & (gray <= thresh[1])] = 1
    return binary

In [124]:
def rgb2gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [125]:
# RGB to indivisual channels: R, G, B
def rgb2R(image):
    R = image[:,:,0]
    return R
    
def rgb2G(image):
    G = image[:,:,1]
    return G
    
def rgb2B(image):
    B = image[:,:,2]
    return B

In [126]:
# HLS to indivisual channels H, L, S
def rgb2H(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    H = image[:,:,0]
    return H
    
def rgb2L(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    L = image[:,:,1]
    return L
    
def rgb2S(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = image[:,:,2]
    return S

In [127]:
def sBinary(image):
    S = rgb2S(image)
    thresh = (90, 255)
    S_binary = np.zeros_like(S)
    S_binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return S_binary

def hBinary(image):
    H = rgb2H(image)
    thresh = (15, 100)
    H_binary = np.zeros_like(H)
    H_binary[(H > thresh[0]) & (H <= thresh[1])] = 1
    return H_binary

def lBinary(image):
    L = rgb2L(image)
    thresh = (15, 100)
    L_binary = np.zeros_like(L)
    L_binary[(L > thresh[0]) & (L <= thresh[1])] = 1
    return L_binary

### Converting Images to a video

# Editing Pipeline

In [128]:
def editing_pipeline(frames_dir):    
    frames = glob.glob(frames_dir + "/" + "*.jpg")
    frames = natsorted(frames)
    #print(frames)
    print("Creating a edited frame saving folder if it is not present\n")
    
    edited_frames_dir = frames_dir + "_edited_frames"
    
    # Folder creation for frames
    CHECK_FOLDER = os.path.isdir(edited_frames_dir)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(edited_frames_dir)
        
    fnum = 0
    n = len(frames)
    e = 9
    print("Editing...")
    for frame_name in tqdm(frames):
        frame = mpimg.imread(frame_name)
        if fnum < n / e:
            frame = Canny(frame)
        
        elif fnum < 2 * n / e:
            frame = colorBinary(frame)
        
        elif fnum < 3 * n / e:
            frame = rgb2gray(frame)
            
        elif fnum < 4 * n / e:
            frame = rgb2gray(frame)
            
        elif fnum < 5 * n / e:
            frame = rgb2R(frame)
        
        elif fnum < 6 * n / e:
            frame = rgb2G(frame)
        
        elif fnum < 7 * n / e:
            frame = rgb2B(frame)
        
        elif fnum < 8 * n / e:
            frame = rgb2H(frame)
        
        elif fnum < 9 * n / e:
            frame = rgb2L(frame)
        
        
        cv2.imwrite(edited_frames_dir + "/" + str(fnum) + ".jpg", frame)
        fnum += 1
    
    print("Editing Complete!!!")
    return edited_frames_dir
    

In [4]:
# Load video and convert to frames
### Takes Time ###
video_name = "123.mp4"
frames_dir = vid2frame(video_name)

123_frames
Creating a folder if it is not present

Created the folder!
Converting to Frames....
Video to frame Conversion Complete.


In [114]:
edited_frames_dir = editing_pipeline(frames_dir)

  0%|                                                                                 | 3/4505 [00:00<03:12, 23.44it/s]

Creating a edited frame saving folder if it is not present

Editing...


100%|██████████████████████████████████████████████████████████████████████████████| 4505/4505 [03:34<00:00, 21.00it/s]

Editing Complete!!!


In [5]:
fps = 2
#frame2vid(edited_frames_dir, fps)

frame2vid(frames_dir, fps)

  4%|██▊                                                                             | 17/480 [00:00<00:02, 163.85it/s]

Joining frames...


 19%|███████████████▎                                                                | 92/480 [00:00<00:00, 436.65it/s]

Saving to a video...


100%|███████████████████████████████████████████████████████████████████████████████| 480/480 [00:01<00:00, 471.90it/s]

Video Saved
